# Project Lab #2: Part 2
## Analyses of Subduction Zone Seismicity


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import earthquake_fns as eq
import math
from datetime import datetime
import warnings

___ 

## A) Write the following functions and add them to your module `earthquake_fns`


### Function 1:  `select_quake_subset` 

<div class="alert alert-block alert-info">
    
**Note**:  There is more than one way to write the body of code for this function, so don't worry if your approach is different from that of another group.  We give you some checks below so you can verify your function is working correctly.
   
</div>

**Code:** The body of `select_quake_subset` extract the subset of an earthquake `DataFrame` that is specified by any combination of (1) geographical region, (2) a time window, (3) a depth range and (4) a magnitude range.  The function should be inclusive of the edges of any selection.  


**Usage:**
```python
df_sub = select_quake_subset(
   df,
   times = [min_time, max_time],
   lons = [min_lon, max_lon],
   lats = [min_lat, max_lat],
   depths = [min_depth, max_depth],
   mags = [min_mag, max_mag]
)
```
**Inputs:** 
> - `df`: an IRIS earthquake `DataFrame` (from Part 1 of the Project lab).
> - `times`: a 2-element list, array, or tuple, e.g. `[start_time, end_time]` containing the start and end times **in that order** for the time window to subset.
> - `lons`: a 2-element list, array, or tuple, e.g. `[min_lon, max_lon]` containing the western, eastern **in that order** of the longitudes interval to subset.
> - `lats`: a 2-element list, array, or tuple, e.g. `[min_lat, max_lat]` containing the southern and northern boundaries **in that order** of the latitudes interval to subset.
> - `depths`: a 2-element list, array, or tuple, e.g. `[min_depth, max_depth]` containing the minimum and maximum depths **in that order** for the depth interval to subset.
> - `mags`: a 2-element list, array, or tuple, e.g. `[min_mag, max_mag]` containing the minimum and maximum magnitudes **in that order** for the magnitude interval to subset.

<br> 

**Default input arguments:** Add default arguments for inputs `times`, `lons`, `lats`, `depths`, and `mags` so that you can call the function `select_quakes_subset` by specifying only the input for `df`. The function call `subselect_quake_subset(df)` should return the original dataframe without applying any subselection. Design your function so that when an optional argument provided, the function will apply a subselection using that argument. e.g. `subselect_quake_subset(df, lats=[-90, 0])` will return a dataframe that contains earthquakes in the Southern hemisphere only, without applying any subselection by longitudes, depths, magnitudes or times. 


**Outputs:** 
> - *df_sub*: a `DataFrame` subselected from the input `df`, which contains the subset of data that meets the selection criteria provided in the input arguments.

<br> 

### Function 2:  `get_slope`

**Code:** The body of `get_slope` should calculate the slope of a line in *degrees*, given by
$ m = \frac{rise}{run} \equiv \frac{\Delta y}{\Delta x}$ (see Background Reading), between a starting point and ending point, each specified by an $x, y$ coordinate pair in units of kilometres.


**Usage:**
```python
   slope = get_slope(start_pt, end_pt) 
```

**Inputs:** 
> - `start_pt`: an array, list, or tuple containing the x- and y- coordinates (in km) of the starting point $(x_1, y_1)$.  
> - `end_pt`: an array, list, or tuple containing the x- and y- coordinates (in km) of the starting point $(x_2, y_2)$.

**Error Checking / Exceptions**. 
> -  The code should raise a `ValueError` exception if division by zero occurs in the slope calculation. Include a helpful error message when the exception is raised, specifying that if the x-coordinate of the start and end point cannot be the same. Do *not* raise a bare exception without a custom message. 

**Outputs:** 
> - *slope*: slope between the start and end points, in *degrees*.

--------------
## B. Getting Set Up

-  Open a new notebook and import your module `earthquake_fns`, along with `numpy`, `matplotlib.pyplot`, `datetime` and `pandas`.

- Add the necessary code from **Part 1** to: 
> - load the coastlines data set using `get_coastlines`
> - load the plate boundaries file using `get_plate_boundaries`
> - load the earthquake data using `get_earthquakes`

- Test your function  `select_quake_subset` using the dataframe produced by `get_earthquakes` for different choices of subsetting your data.
 >  e.g. you can test it by repeating the subselection in **C3** of **Part 1** and checking you get the same number of earthquakes in 2022. 
> 
>  **Additional checks:**   
> - There were 3166 quakes at depths $\geq$ 600 km.
> - There were 96598 quakes within 10° latitude of the equator (ie. from 10°S to 10°N). 
> - There were 1540 quakes in year 2000 with longitudes $\leq$ 0,  latitudes $\geq$ 0, and with depths $\leq$ 100 km.
> - There were 324 quakes between magnitudes 7 and 8 since 2000. 


In [ ]:
warnings.filterwarnings("ignore", category=UserWarning) #FRANCIS SAID IT WAS FINE
longitudes, latitudes = eq.get_coastlines("./m_coasts.csv")
pb_dict = eq.get_plate_boundaries("./all_boundaries.csv")
earthquakes = eq.get_earthquakes("./IRIS_eq_010100_112422_mag4.csv")
lats, lons, depths, magnitudes, times = eq.parse_earthquakes_to_np(earthquakes)
test = eq.select_quake_subset(earthquakes, 
                    [np.min(times), np.max(times)], 
                    [np.min(lons), np.max(lons)], 
                    [np.min(lats), np.max(lats)],
                    [np.min(depths), np.max(depths)],
                    [np.min(magnitudes), np.max(magnitudes)])

___ 



## C:  Seismicity along the Peru-Chile Trench

The ongoing subduction of the Nazca Plate beneath the overlying South American Plate long the Peru-Chile Trench is responsible for the formation of the Andes mountains and the majority of the earthquakes in the region. The region between $-90^\circ$E to $-60^\circ$E,  and $-30^\circ$N to $-5^\circ$N used for this part is centered on the western coast of South America and the Peru-Chile Trench where the Nazca Plate meets the South American Plate. 


### C1. Large-scale observations of South American earthquakes.

**C1.1)** Subselect two new dataframes in the region between $-90^\circ$E to $-60^\circ$E,  and $-30^\circ$N to $-5^\circ$N using calls to your function `select_quake_subset`. 
> - a) Shallow earthquakes: this dataframe should contain shallow (< 50 km depth) earthquakes in this region
>   
> - b) Deep earthquakes: this dataframe should contain deep (>= 50 km depth) earthquakes in this region.

**Note**: You can at any stage convert the dataframes into numpy arrays of *longitude, latitude, depth, magnitude and time* by using two calls to your function `parse_earthquakes_to_np`. 


<br>

**C1.2)** **Figure 1**: Make a new map figure in the same geographical region defined in **C1.1**. 
> - Scatter the locations of the shallow earthquakes using gray markers.
>  
> - Scatter the locations of the deep earthquakes as filled circles coloured by depth. 
>  
> - Scatter the plate boundaries using markers that are circles, ensuring they are large enough and have sufficient contrast to be seen.
>  
> - Plot the coastlines as dark gray lines. 
> 
> - Adjust the aspect ratio of your plot so that the map is not distorted (something close to 1:1).
> 
> -  The plate boundary locations in your figure will show that there are two major tectonic plates in the region. The large plate to the west is the Nazca Plate, and the large one to the east (including the continent of South America) is the South American Plate. The small plate near the centre of the map is the Altiplano plate. Annotate your map with the plate names/abbreviations.
> 
> -  Include a colorbar for depths, informative labels and a title.   


<br>

**C1.3)** **Written answer**: 
> Make a Markdown cell at the end of your notebook with header **## Observations and Discussion** and make **3 observations** about the relationship of shallow and deep seismicity in this region with respect to the plate boundaries and write them in a cell at *the very end of your notebook* (you can copy and paste the template cell at the end of this notebook into your notebook).


In [ ]:
peru_chile_trench_shallow = eq.select_quake_subset(earthquakes, 
                                        [np.min(times), np.max(times)], 
                                        [-90, -60], 
                                        [-30, -5],
                                        [np.min(depths), 49],
                                        [np.min(magnitudes), np.max(magnitudes)])
peru_chile_trench_deep = eq.select_quake_subset(earthquakes, 
                                        [np.min(times), np.max(times)], 
                                        [-90, -60], 
                                        [-30, -5],
                                        [50, np.max(depths)],
                                        [np.min(magnitudes), np.max(magnitudes)])
lats_s, lons_s, depths_s, mag_s, times_s = eq.parse_earthquakes_to_np(peru_chile_trench_shallow)
lats_d, lons_d, depths_d, mag_d, times_d = eq.parse_earthquakes_to_np(peru_chile_trench_deep)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(longitudes, latitudes, color='dimgrey', linewidth=1)

#do we need to plot the nazca plate boundary? I can do it

colour_by_depth_shallow = (depths_s - np.min(depths_s)) / (np.max(depths_s) - np.min(depths_s)) #colouring by depth
colour_by_depth_deep = (depths_d - np.min(depths_d)) / (np.max(depths_d) - np.min(depths_d))

alphas_s = 1 - colour_by_depth_shallow
alphas_d = 1 - colour_by_depth_deep

ax.scatter(lons_s, lats_s, color = 'grey', s = 5, alpha = alphas_s)
ax.scatter(lons_d, lats_d, color = 'blue', s = 5, alpha = alphas_d) 
ax.set_xlim(-120, -30)
ax.set_ylim(-45, 10)

In [ ]:
nazca_plate = eq.select_quake_subset(earthquakes, 
                                  [np.min(times), np.max(times)], 
                                  [-75, -60], 
                                  [-25, -22], 
                                  [np.min(depths), np.max(depths)],
                                  [np.min(magnitudes), np.max(magnitudes)])
lats, lons, depths, mag, times = eq.parse_earthquakes_to_np(nazca_plate)

fig, ax = plt.subplots(figsize=(10, 5))

ax.scatter(lons, depths * -1, c = "black", alpha = 0.5, s = 5)
ax.scatter(-71.4 , 0, marker = "*", s = 100, c= "red")
ax.set_xlabel("Longitude (degrees)")
ax.set_ylabel("Depth (km)")
ax.set_title("Earthquake Depth versus Longitude")

### C3:  Computing the subduction angle (dip) of the Nazca Plate


<div class="alert alert-block alert-warning">
    
For this section see background reading **Subducting Slab Geometry** and **Concepts from Earlier Labs**. You will also use the *subset of data generated in C2.*
    
</div>

**C3.1)** Convert the earthquake longitudes into distances (in kilometres) from the longitude of the mean plate boundary.  Write these distances to an array named e.g., *distances*. Confirm that your array has the same number of elements as the longitudes being converted.


**C3.2)** **Figure 3:**
> - Remake the longitude-depth plot (*Figure 2*) but now as a distance-depth plot (i.e. the x-axis should be distance from the mean longitude of the plate boundary, ie. array `distances`).
> -  Set the aspect of this plot equal so that 1 km on the x-axis occupies the same length as 1 km on the y-axis.
> -  Include the necessary title, colorbar, and labels where necessary. 

<br>

**C3.3)** Compute the piece-wise slope of the subducting slab in two sections (you will calculate two slopes).  To do this you will need to estimate the  $\Delta y$ or the depth range of the quakes to which you want to fit a straight line, and the horizontal distance in km spanned by those quakes, $\Delta x$.  Do this as follows:
>  a) Get the **median depth** and **median distance** from the plate boundary for the S. American earthquakes that meet each of the following selection criteria:
> -  i) Depths < 20 km and distances < 50 km.  Call this distance, depth pair $(x_1, y_1)$.
> - ii) Depths 140 - 150 km.  Call this distance, depth pair $(x_2, y_2)$.
> - iii) Depths > 500 km.  Call this distance, depth pair $(x_3, y_3)$.


> b) Plot these three $(x, y)$ pairs as heavy red `+` signs on your distance-depth plot, and connect the $(x, y)$ pairs with a fine dashed/dotted line to help visualize the slope.

> c) Compute the slopes between $(x_1, y_1)$ and $(x_2, y_2)$ points by calling your function `get_slope()`  and repeat for the $(x_2, y_2)$ and $(x_3, y_3)$ points.  Print your results to the screen in a clear, informative way. Also annotate the dashed/dotted lines on your figure with your calculated slopes.


<br> 

**C3.4)** **Written answer:** Answer the following in the **Observations and Discussion** Markdown cell at the end of your notebook:  
> Do your quantitative estimates of slope (sign, magnitude) make sense compared with what is in your longitude-distance plot?
> Include a short discussion about your results and anything you find interesting or puzzling in the data and or slope calculations (2-3 sentences maximum).


In [ ]:
one_longitude = 2 * np.pi * 6371 * np.cos(np.deg2rad(np.mean(lats)))/ 360
distances = (lons - (-71.4)) * one_longitude
#print(lons.shape, distances.shape)

s_american_earthquakes_select_i = eq.select_quake_subset(nazca_plate, 
                                      [np.min(times), np.max(times)], 
                                      [np.min(lons), np.max(lons)], 
                                      [np.min(lats), np.max(lats)], 
                                      [np.min(depths), 20],
                                      [np.min(magnitudes), np.max(magnitudes)])
lats_i, lons_i, depths_i, mag_i, times_i = eq.parse_earthquakes_to_np(s_american_earthquakes_select_i)

dist_i = np.where(((lons_i - (-71.4)) * one_longitude) <= 50)[0]

med_i = (np.median(depths_i), np.median((lons_i[dist_i] - (-71.4)) * one_longitude))
x_1 = med_i[0]
y_1 = med_i[1]

#==============================================================================
s_american_earthquakes_select_ii = eq.select_quake_subset(nazca_plate, 
                                      [np.min(times), np.max(times)], 
                                      [np.min(lons), np.max(lons)], 
                                      [np.min(lats), np.max(lats)], 
                                      [140, 150],
                                      [np.min(magnitudes), np.max(magnitudes)])
lats_ii, lons_ii, depths_ii, mag_ii, times_ii = eq.parse_earthquakes_to_np(s_american_earthquakes_select_ii)
med_ii = (np.median(depths_ii), np.median((lons_ii - (-71.4)) * one_longitude))
x_2 = med_ii[0]
y_2 = med_ii[1]

#==============================================================================
s_american_earthquakes_select_iii = eq.select_quake_subset(nazca_plate, 
                                      [np.min(times), np.max(times)], 
                                      [np.min(lons), np.max(lons)], 
                                      [np.min(lats), np.max(lats)], 
                                      [500, np.max(depths)],
                                      [np.min(magnitudes), np.max(magnitudes)])
lats_iii, lons_iii, depths_iii, mag_iii, times_iii = eq.parse_earthquakes_to_np(
    s_american_earthquakes_select_iii)
med_iii = (np.median(depths_iii), np.median((lons_iii - (-71.4)) * one_longitude))
x_3 = med_iii[0]
y_3 = med_iii[1]
#print(med_i, med_ii, med_iii)

slope_1to2 = eq.get_slope(med_i, med_ii)
slope_2to3 = eq.get_slope(med_ii, med_iii)
print(f'The Angle of the Slope between Subset 1 and Subset 2 is {slope_1to2:.2f} degrees.') #just change the rounding to fucking whatever
print(f'The Angle of the Slope between Subset 3 and Subset 3 is {slope_2to3:.2f} degrees.')

In [ ]:
fig, ax = plt.subplots()
ax.scatter(distances, depths * -1, c = "black", alpha = 0.10) #flipped
ax.scatter(x_1 , (y_1 * -1), marker = "+", s = 100, c= "red")
ax.scatter(x_2 , (y_2 * -1), marker = "+", s = 100, c= "red")
ax.scatter(x_3 , (y_3 * -1), marker = "+", s = 100, c= "red")

ax.plot([x_1, x_2, x_3],
        [(y_1 * -1), (y_2 * -1), (y_3 * -1)],
        linestyle='--', color='black')
ax.annotate(f'Slope: {slope_1to2:.2f} degrees.', xy=(75, -250), xytext=(-200, -500),
                 arrowprops=dict(facecolor='black', arrowstyle='->', connectionstyle='arc3,rad=0.3'), fontsize = 8)

ax.annotate(f'Slope: {slope_2to3:.2f} degrees.', xy=(300, -500), xytext=(600,-450),
                 arrowprops=dict(facecolor='black', arrowstyle='->', connectionstyle='arc3,rad=0.3'), fontsize = 8)
ax.set_xlabel("Distance (km)")
ax.set_ylabel("Depth (km)")
ax.set_title("Earthquake Depth versus Distance From Mean Plate Longitude")
ax.set_xlim(-250, 1000)
ax.set_ylim(-1000, 1000)
plt.show()

<div class="alert alert-block alert-success">

# Submission Instructions

###  You should submit: 
- Your python file `earthquake_fns.py` that is the module containing your functions form Parts 1 and 2
    
- A single Jupyter notebook with ONLY 3 cells with contents as follows
> - Cell 1: A markdown cell with your names and student numbers
> - Cell 2: A code cell with your import statements, function calls, and code to produce the requested figures / analyses for **Section C**. 
> - Cell 3: A markdown cell with your observations and discussion (you can cut and paste the one below and fill it in).

- A `.pdf` file of your Jupyter Notebook which includes any output requested (figures, print statements, etc.), but no other output (i.e. no debugging checks etc).

</div>

## Observations and Discussion


### From C1.3:  Three observations from your map (shallow and deep seismicity along western S. America):

<br><br><br>




### From C2.3:  Three observations from your longitude-depth plot

<br><br><br>





### From C3.4: Discussion (2-3 sentences max) of results from your calculations of dip of the subducting slab:

<br><br><br>





C1.3:
Shallow earthquakes are far more widely distributed. Though they are mainly focused on the plate boundaries, they are sporadically distributed.
By comparison, deeper earthquakes are far more centrally localized and make up the borders of the plate boundary.
Generally, it appears that the deeper subset is far more frequent, suggesting high magnitudes of seismic activity.

C2.3:
Depth increases as longitude moves away from the average plate boundary. Frequency also increases as longitude moves away from the average plate boundary.
Linearity of both the depth-longitude and frequency-longitude relationships suggest that there should be data between -66 and -64. There may be some external factor preventing seismic activity in that area.

C3.4:

Yes, this makes sense. A steeper subduction plate is generally correlated with shallower earthquake depths, while a less steep angle is correlated with deeper earthquake depths. This is reflected in the plot, where the trend of earthquake depths increases in the x-axis area of the shallower slope. Curiously, steeper slopes theoretically lead to higher seismic activity and more frequency earthquakes, but this plot represents the opposite effect.